When training neural networks, the most frequently used algorithm is `back propagation`. In this algorithm, parameters (model weights) are adjusted according to the `gradient` of the loss function with respect to the given parameter. The loss function calculates the difference between the expected output and the actual output that a neural network produces. The goal is to get the result of the loss function as close to zero as possible. The algorithm traverse backwards through the network network to adjust the weights and bias to retrain the model. That's why it's called back propagation. This back and forward process of retraining the model over time to reduce the loss to 0 is called the gradient descent.

In [4]:
import torch

In [5]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print('loss: ', loss)

loss:  tensor(0.9560, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


X * W + b = z  -> CE(y) -> Loss


W 和 b 是需要优化的参数，因此，我们需要能够计算关于这些变量的损失函数的梯度。 为了做到这一点，我们将设置 `requires_grad` 标志。实际上计算梯度的是一类 `Function` 的对象。该对象知道如何在正向（`forward`）计算函数，以及如何在反向传播（`backward propagation`）步骤中计算其导数。对反向传播函数的引用存储在张量的 grad_fn 属性中。

In [6]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x000001D2443C3B50>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001D2443C1ED0>


实际上我们是在 x 和 y 的一些固定值下，计算$\frac{\partial loss}{\partial w}$ and$\frac{\partial loss}{\partial b}$，为了计算这个导数，我们需要调用 `loss.backward()`，然后我们需要获取 w.grad 和 b.grad。

In [7]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2850, 0.1462, 0.1008],
        [0.2850, 0.1462, 0.1008],
        [0.2850, 0.1462, 0.1008],
        [0.2850, 0.1462, 0.1008],
        [0.2850, 0.1462, 0.1008]])
tensor([0.2850, 0.1462, 0.1008])


默认情况下，只要设置了 `requires_grad=True` 的 `tensor` 都会跟踪它的计算历史，并且允许我们通过调用 `.backward()` 来自动计算梯度。这个 tensor 的梯度将会累计到 `.grad` 属性中。但是并不是所有情况都需要这样做，我们可以通过 `torch.bo_grad():` 来停止 tensor 历史记录的跟踪。

In [8]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


另外一种方式就是使用 `detach()`。

In [9]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


`Torch` 是使用`计算图`去计算梯度的。`autograd` 将保存了数据的记录和所有执行过的操作都存在一个由函数组成的 `DAG`（Directed Acyclic Graph） 中。在 `DAG` 中，叶子节点是输入的 `tensor`，根节点是输出的 `tensor`。通过从根节点到叶子节点跟踪此图，可以使用链式法则自动计算梯度。

在前向传播过程中，`autograd` 会同时做两件事：
- 执行请求的操作，以计算结果 `tensor`，
- 维护 `DAG` 中，操作的`梯度函数`。

当 `DAG` 的根节点的 `.backward()` 方法被调用时，会执行反向传播，`autograd` 会：
- 从每个 `.grad_fn` 属性中计算梯度，
- 将它们累加到各自的 `tensor` 的 `.grad` 属性中，
- 使用链式法则，一直传播到叶子节点。

需要注意的重要一点是该图是从头开始重新创建的；在每次 .backward() 调用之后，autograd 开始填充一个新图。这正是允许使用控制流语句的原因；我们可以在每次迭代时改变形状，大小和操作，因为这些都会被记录在图中。